In [ ]:
from scipy.interpolate import interp1d
from scipy.ndimage.interpolation import rotate
from sklearn.preprocessing import normalize
import numpy as np
import matplotlib.pyplot as plt
import trimesh
import pyembree
import random

from Morphology import Morphology
from Fibril import Fibril

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
# TO DO:
# Core-shell cylinders
# 

In [ ]:
#https://stackoverflow.com/questions/53999426/how-to-parameterize-a-curved-cylinder

In [ ]:
# Declare model box size in nm (x,y,z)
x_dim_nm  = 1024
y_dim_nm  = 1024
z_dim_nm  = 128
pitch_nm = 2 # Dimension of voxel in nm

# Initialize morphology
morphology = Morphology(x_dim_nm, y_dim_nm, z_dim_nm, pitch_nm, 2)
morphology.set_model_parameters(radius_nm_avg = 15.,
                                radius_nm_std = 1.0,
                                max_num_fibrils = 250,
                                fibril_length_range_nm = [100, 400])

# morphology.fill_model()

In [ ]:
scene = morphology.get_scene(show_bounding_box=True)
# scene.show()

In [ ]:
scene = morphology.get_scene(show_bounding_box=True, show_voxelized=True)
# scene.show()

In [ ]:
indices = np.array([[0,0,0]],dtype=int)

fibrils = morphology.fibrils
for fibril in fibrils:
    voxel_mesh = fibril.voxel_mesh
    indices = np.append(indices, np.array(voxel_mesh.vertices, dtype=int),axis=0)

indices  = [index for index in indices if index[0] < morphology.x_dim and index[1] < morphology.y_dim and index[2] < morphology.z_dim]
    
voxel_box = np.zeros((morphology.x_dim, morphology.y_dim, morphology.z_dim))
for index in indices:
    voxel_box[tuple(index)] = 1

In [ ]:
plt.close()
# plt.figure(dpi=200)
# plt.imshow(voxel_box[:,:,11])

In [ ]:
fibril_volume = 0
fibril_vox_volume = 0
for fibril in morphology.fibrils:
    fibril_volume     += fibril.volume
    fibril_vox_volume += fibril.voxel_volume
print(fibril_volume)
print(f'Fibril Volume    : {fibril_volume:.2f}')
print(f'Fibril Vox Volume: {fibril_vox_volume:.2f}')
print(f'Box Volume: {morphology.box_volume}')
mpc = fibril_volume / morphology.box_volume * 100
vpc = fibril_vox_volume / morphology.box_volume * 100
print(f'Mesh Percent Crystallinity: {mpc:.2f}')
print(f'Voxel Percent Crystallinity: {vpc:.2f}')

- Animation for morphology
- List of model parameters

In [ ]:
ma = morphology.mesh_list[0]
va = ma.voxelized(pitch=pitch)

mb = morphology.mesh_list[1]
vb = mb.voxelized(pitch=pitch)

In [ ]:
vmesh_list = []
for mesh in tqdm(morphology.mesh_list):
    v = mesh.voxelized(pitch=1)
    vmesh = v.fill().as_boxes()
    vmesh_list.append(vmesh)

vscene = trimesh.Scene(vmesh_list)
vscene.show()

In [ ]:
vscene = trimesh.Scene(vmesh_list)
vscene.show()

In [ ]:
indices = np.array([[0,0,0]],dtype=int)
for vmesh in vmesh_list:
    indices = np.append(indices, np.array(vmesh.vertices, dtype=int),axis=0)
# np.shape(indices)
indices  = [index for index in indices if index[0] < morphology.x_dim and index[1] < morphology.y_dim and index[2] < morphology.z_dim]
    
voxel_box = np.zeros((morphology.x_dim, morphology.y_dim, morphology.z_dim))
for index in indices:
    voxel_box[tuple(index)] = 1

In [ ]:
vmesh_list[0].vertices

In [ ]:
np.any(vmesh_list[0].vertices > 500)

In [ ]:
plt.figure()
plt.imshow(voxel_box[:,:,9])
# np.where(voxel_box == 1)

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
ax.voxels(voxel_box)
ax.set_aspect('equal')
plt.show()

In [ ]:
# print(vmesh_list)
# vmesh_list.append(morphology.bounding_path)
vscene = trimesh.Scene(vmesh_list)
vscene.show()

In [ ]:
plt.imshow(v.matrix[:,10])
plt.show()

In [ ]:
v.translation

In [ ]:
vmesh_list.pop()

### What's next?
###### - Get the angles sorted
###### - Functionality to check average angle
###### - Check degree of crystallinity
###### - Voxelize mesh
###### - Angles 

In [ ]:
full_cyl_vox = full_cyl_mesh.voxelized(pitch=2)
full_cyl_vox.fill()
full_cyl_vox_mesh  = full_cyl_vox.as_boxes()
print(full_cyl_vox.volume)
print(full_cyl_mesh.volume)

In [ ]:
full_cyl_vox_mesh.show()

In [ ]:
broken = trimesh.repair.broken_faces(full_cyl_vox_mesh, color=[255, 0, 0, 255])
print(len(broken))
print(full_cyl_vox_mesh.is_watertight)
full_cyl_vox_mesh.show(smooth=False)

In [ ]:
core_cyl_vox = core_cyl_mesh.voxelized(pitch=2)
core_cyl_vox.fill()
core_cyl_vox_mesh  = core_cyl_vox.as_boxes()
print(core_cyl_vox.volume)
print(core_cyl_mesh.volume)

In [ ]:
shell_cyl_vox = shell_cyl_mesh.voxelized(pitch=2)
shell_cyl_vox.fill()
shell_cyl_vox_mesh = shell_cyl_vox.as_boxes()
print(shell_cyl_vox.volume)
print(shell_cyl_mesh.volume)

In [ ]:
within_mesh(full_cyl_vox_mesh, core_cyl_vox_mesh)

In [ ]:
shell_cyl_mesh.is_watertight

In [ ]:
v1_mesh.visual.face_colors[:] = [255, 0, 0, 0]
trans_matrix = trimesh.transformations.translation_matrix([64, 0, 0])
v2_mesh.apply_transform(trans_matrix)
voxel_scene = trimesh.Scene([v3_mesh, m3])
voxel_scene.show()

In [ ]:
v

In [ ]:
radius = 10
height = 40
pitch  = 0.2
mesh = trimesh.primitives.Cylinder(radius=radius, height=height, use_embree=True)
mesh_vol = mesh.volume
print(f'Mesh Volume: {mesh_vol:.2f}')

voxel_grid = mesh.voxelized(pitch=pitch)
voxel_mesh = voxel_grid.fill().as_boxes()
voxel_mesh_vol = voxel_mesh.volume
print(f'Voxel Mesh Volume: {voxel_mesh_vol:.2f}')

vol_diff = np.abs(voxel_mesh_vol - mesh_vol)/mesh_vol * 100
print(f'Volume Difference: {vol_diff:.2f}%')

In [ ]:
import numpy as np

In [ ]:
mu, sigma = 0, 1.0
samples   = 100000
gnoise = np.random.normal(mu, sigma, samples)
enoise = np.random.uniform(0.0, np.pi, samples)
abs(mu - np.mean(gnoise))

theta = (90 + gnoise)/180 * np.pi
phi    = enoise

In [ ]:
vec2 = np.sin(theta)*np.cos(phi)
vec1 = np.sin(theta)*np.sin(phi)
vec0 = np.cos(theta)
plt.figure()
count, bins, ignored = plt.hist(vec2, 30, density=True)
plt.title('vec2')
plt.figure()
count, bins, ignored = plt.hist(vec1, 30, density=True)
plt.title('vec1')
plt.figure()
count, bins, ignored = plt.hist(vec0, 30, density=True)
plt.title('vec0')
plt.show()